In [1]:
import pandas as pd
from datetime import date, timedelta
from random import randint
from time import sleep
import os
import numpy as np
import pymysql
from sqlalchemy import create_engine
import joblib
import requests
from bs4 import BeautifulSoup
import pulp

## TODO

No todo's right now.

In [2]:
## Variables
year = 2025

# Get today's date
today = date.today()
season_start_date = '2024-10-04'

In [3]:
username = "root"
password = "Sp1d3rman"
host = "localhost"
port = "3306"
database = "nhl_optimizer"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [4]:
query = f"""
SELECT *
FROM player_data
WHERE Date >='{season_start_date}'
"""

df = pd.read_sql(query,engine)

In [5]:
# Clean the data
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates()

df = df.sort_values(by=['Player','Date'])

In [6]:
# Compute rolling aggregates

# Define the number of games for the rolling window
span = 10 # adjust as needed
min_periods = 5

# Group by 'Player' and apply rolling calculations
grouped = df.groupby('Player',as_index=False,group_keys=False)

stats = ['GP','TOI', 'TOI/GP', 'Goals/60',
       'Total Assists/60', 'First Assists/60', 'Second Assists/60',
       'Total Points/60', 'IPP', 'Shots/60', 'SH%', 'ixG/60', 'iCF/60',
       'iFF/60', 'iSCF/60', 'iHDCF/60', 'Rush Attempts/60',
       'Rebounds Created/60', 'PIM/60', 'Total Penalties/60', 'Minor/60',
       'Major/60', 'Misconduct/60', 'Penalties Drawn/60', 'Giveaways/60',
       'Takeaways/60', 'Hits/60', 'Hits Taken/60', 'Shots Blocked/60',
       'Faceoffs Won/60', 'Faceoffs Lost/60','Faceoffs %']

for stat in stats:
    df[stat] = grouped[stat].apply(lambda x: x.ewm(span=span,min_periods=min_periods).mean())
    
latest_stats = df.groupby('Player').tail(1)


final_df = latest_stats[['Player','Team','Position'] + stats].reset_index(drop=True)


### Player Point Value Data

In [7]:
pv_df = pd.read_csv('nhl_players.csv')

## Load and Process Models

In [8]:
# Load the models
loaded_goals_pipeline = joblib.load('goals_model.pkl')
loaded_assists_pipeline = joblib.load('assists_model.pkl')

# Read and process the data
# season_totals = pd.read_csv('season_totals_24.csv',index_col=0)
# season_totals.drop(columns=["Goals/60","Total Assists/60"], inplace=True)

season_totals = final_df.drop(columns=['Goals/60','Total Assists/60'])


# # Predictions
goal_predictions = loaded_goals_pipeline.predict(season_totals)
assist_predictions = loaded_assists_pipeline.predict(season_totals)

## Functions

In [9]:
from datetime import date, timedelta

def fetch_game_data(year: int):
    """
    Fetches the dates of NHL games and teams involved for a given year.
    
    Parameters:
    - year: The year for which you want to fetch game data.

    Returns:
    - DataFrame with dates, visitor, and home teams.
    """
    
    url = f"https://www.hockey-reference.com/leagues/NHL_{year}_games.html"
    dfs = pd.read_html(url)
    df = dfs[0]
    
    # Convert the "Date" column to datetime.date format
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Extract the relevant columns
    game_data = df[["Date", "Visitor", "Home"]]

    return game_data

def filter_dates_for_week(dates, start_date):
    """
    Filters the provided dates for those that fall within the week starting at start_date.
    
    Parameters:
    - dates: List of dates.
    - start_date: The starting date of the week. 

    Returns:
    - A list of dates that fall within the desired week.
    """
    # Convert start_date to a date object if it's a string
    if isinstance(start_date, str):
        start_date = date.fromisoformat(start_date)
    
    # Calculate the end date of the week
    end_date = start_date + timedelta(days=6)

    # Filter the dates
    week_dates = [d for d in dates if start_date <= d <= end_date]
    
    return week_dates

def opponents_for_team_for_week(year, start_date):
    """
    Fetches the opponents each team faces within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and lists of opponents as values.
    """
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Prepare opponent information
    visitor_opponents = week_games.set_index("Visitor")["Home"].to_dict()
    home_opponents = week_games.set_index("Home")["Visitor"].to_dict()
    
    opponents = {}
    for team in set(week_games["Visitor"].tolist() + week_games["Home"].tolist()):
        opponents[team] = list(set([visitor_opponents.get(team, "")] + [home_opponents.get(team, "")]))

    return opponents


#-------#

def calculate_weekly_team_multipliers(year, start_date, points_df):
    """
    Calculate the weekly multipliers for teams based on their opponents for a given week.
    """
    
    # Fetch the opponents for each team for the week
    games_week = opponents_for_team_for_week(year, start_date)

    weekly_multipliers = {}
    for team, opponents in games_week.items():
        multipliers = []
        for opponent in opponents:
            multiplier_row = points_df[points_df['Team Name'] == opponent]
            if not multiplier_row.empty:
                multipliers.append(multiplier_row['multiplier'].iloc[0])
            else:
                print(f"Warning: Couldn't find multiplier for opponent {opponent}. Skipping...")

        weekly_multipliers[team] = sum(multipliers) / len(multipliers) if multipliers else 1  # Defaulting to 1 if no games

    return weekly_multipliers

# Now, you can create a new column in your players DataFrame to store the composite multiplier for the week:


def games_count_for_team_for_week(year, start_date):
    """
    Fetches the number of games each team plays within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and the number of games they play that week as values.
    """
    
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Count the number of games for each team
    visitor_counts = week_games["Visitor"].value_counts().to_dict()
    home_counts = week_games["Home"].value_counts().to_dict()
    
    total_counts = {}
    for team in set(list(visitor_counts.keys()) + list(home_counts.keys())):
        total_counts[team] = visitor_counts.get(team, 0) + home_counts.get(team, 0)

    return total_counts

## Team Name Dict

In [10]:
# TEAM NAMES
team_abbreviations = {
 'Anaheim Ducks':'ANA',
 'Boston Bruins':'BOS',
 'Buffalo Sabres':'BUF',
 'Calgary Flames':'CGY',
 'Carolina Hurricanes':'CAR',
 'Chicago Blackhawks':'CHI',
 'Colorado Avalanche':'COL',
 'Columbus Blue Jackets':'CBJ',
 'Dallas Stars':'DAL',
 'Detroit Red Wings':'DET',
 'Edmonton Oilers':'EDM',
 'Florida Panthers':'FLA',
 'Los Angeles Kings':'L.A',
 'Minnesota Wild':'MIN',
 'Montreal Canadiens':'MTL',
 'Nashville Predators':'NSH',
 'New Jersey Devils':'N.J',
 'New York Islanders':'NYI',
 'New York Rangers':'NYR',
 'Ottawa Senators':'OTT',
 'Philadelphia Flyers':'PHI',
 'Pittsburgh Penguins':'PIT',
 'San Jose Sharks':'S.J',
 'Seattle Kraken':'SEA',
 'St. Louis Blues':'STL',
 'Tampa Bay Lightning':'T.B',
 'Toronto Maple Leafs':'TOR',
 'Utah Hockey Club':'UTA',
 'Vancouver Canucks':'VAN',
 'Vegas Golden Knights':'VGK',
 'Washington Capitals':'WSH',
 'Winnipeg Jets':'WPG'
}

## Injuries

In [11]:
# Fetch the webpage content
url = "https://www.cbssports.com/nhl/injuries/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Lists to hold our parsed data
teams = []
injury_dataframes = []

# Iterate over each TableBaseWrapper
for wrapper in soup.find_all('div', class_='TableBaseWrapper'):
    
    # Find team name
    team_name_tag = wrapper.find('span', class_='TeamName')
    if team_name_tag:
        team_name = team_name_tag.get_text(strip=True)
        teams.append(team_name)

        # Find the injury table for the team
        table = wrapper.find('table', class_='TableBase-table')
        if table:
            # Use pandas to read the table
            df = pd.read_html(str(table))[0]
            df['Team'] = team_name  # Add a column for the team name
            injury_dataframes.append(df)

# Concatenate all dataframes
injuries_df = pd.concat(injury_dataframes, ignore_index=True)

#Clean the player name up


def clean_player_name(name):
    # Split the name using spaces to identify potential last names
    parts = name.split()

    # The last word (or the two last words in the case of names like "van Riemsdyk") is likely our surname
    surname = parts[-2] + " " + parts[-1] if len(parts) > 2 and len(parts[-2]) <= 3 else parts[-1]

    # Split by surname and get the second part
    full_name = name.split(surname, 1)[1].strip()

    return full_name if full_name else surname

injuries_df['Player'] = injuries_df['Player'].apply(clean_player_name)

## Variables

In [12]:
## VARIABLES

## Updated to match new scoring rules for Office Pools
fantasy_goals = 2
fantasy_assists = 1
goaltending_win = 2
ot_so_loss = 0
shutout = 2

# all_dates = fetch_game_dates(year)
# week_dates = filter_dates_for_week(all_dates, today)

#### Win % for SoS Calc

In [13]:
## GATHER WIN % TO USE FOR SOS CALC

dfs = pd.read_html(f"https://www.hockey-reference.com/leagues/NHL_{year}_standings.html", index_col=0)
east_df = dfs[0].reset_index().rename(columns={'index': 'Team Name'})
west_df = dfs[1].reset_index().rename(columns={'index': 'Team Name'})

east_df = east_df.drop([0, 9], axis=0).reset_index(drop=True)
west_df = west_df.drop([0, 9], axis=0).reset_index(drop=True)
east_df['Team Name'] = east_df['Team Name'].str.replace('*', '', regex=False)
west_df['Team Name'] = west_df['Team Name'].str.replace('*', '', regex=False)


east_df = east_df[['Team Name', 'PTS%']]
west_df = west_df[['Team Name', 'PTS%']]

points_df = pd.concat([east_df, west_df], ignore_index=True)
points_df['PTS%'] = points_df['PTS%'].astype(float)


points_df['Abbreviation'] = points_df['Team Name'].map(team_abbreviations)

## Build Reference DataFrame

In [14]:
# Create dataframe for reference

player_df = pd.DataFrame({
    'Team':season_totals['Team'].values,
    'Position':season_totals['Position'].values,
    'Player':season_totals['Player'].values,
    'Projected Goals/60':goal_predictions,
    'Projected Assists/60':assist_predictions,
    'Projected Goals (TOI)':(season_totals['TOI/GP']/60) * goal_predictions,
    'Projected Assists (TOI)':(season_totals['TOI/GP']/60) * assist_predictions,
})

player_df = player_df.merge(points_df, left_on='Team', right_on='Abbreviation', how='left')

player_df = player_df.round(2).sort_values(by='Projected Goals (TOI)', ascending=False)
player_df['Team'] = player_df['Team Name'].str.split(',').str[0].str.strip()

player_df = player_df[['Team','Abbreviation','Player', 'Position','Projected Goals (TOI)', 'Projected Assists (TOI)']]

player_df = player_df.merge(pv_df[["Team","Player","Position","pv"]], 
                     on=["Team",'Player',"Position"], 
                     how='left')

# 3. Select and rename the desired columns
player_df = player_df.rename(columns={
    'Team_x': 'Old Team',
    'Team_y': 'Team',
})

player_df = player_df[['Team', 'Abbreviation', 'Player', 'Position', 'Projected Goals (TOI)', 'Projected Assists (TOI)']]

# Cleaning the team name (remove asterisks and keep only abbreviation, if needed)
points_df['Team'] = points_df['Team Name']

# Calculating multiplier based on points percentage
points_df['multiplier'] = points_df['PTS%'].apply(lambda x: 0.5 / x if x != 0 else 1.8)

In [15]:
weekly_multipliers = calculate_weekly_team_multipliers(year, today, points_df)
games_this_week = games_count_for_team_for_week(year, today)


# Map this to your players DataFrame
player_df['weekly_multiplier'] = player_df['Team'].map(weekly_multipliers)
player_df['games_this_week'] = player_df['Team'].map(games_this_week)

In [16]:
player_df['projected_goals'] = player_df['Projected Goals (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']
player_df['projected_assists'] = player_df['Projected Assists (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']

# Factoring in fantasy points value for goals and assists
player_df['proj_fantasy_pts'] = (player_df['projected_goals'] * fantasy_goals) + (player_df['projected_assists'] * fantasy_assists)

# Create simple F or D position
player_df.loc[player_df['Position'] != 'D', 'Position'] = 'F'

In [17]:
# Now, merge on 'Player'
merged_df = pd.merge(player_df, injuries_df[['Player', 'Injury Status']], 
                     on='Player', 
                     how='left')

# Set a new column to indicate if a player is injured or not
merged_df['Injured'] = ~merged_df['Injury Status'].isnull()

# Set the projected fantasy points to 0 for injured players
merged_df['proj_fantasy_pts'] = np.where(merged_df['Injured'], 0, merged_df['proj_fantasy_pts'])

In [18]:
projections_df = merged_df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

## Add Team Goaltending Calcs

In [19]:
def estimate_team_goaltending_points(weekly_multipliers, games_this_week, win_points, ot_loss_points, shutout_bonus, avg_ot_loss_freq=0.1, avg_shutout_freq=0.05):
    """
    Estimate the team goaltending points for the week using weekly multipliers, 
    number of games this week, and the fantasy points system.

    Returns:
    - A dictionary with team abbreviations as keys and a tuple of projected goaltending points and games_this_week as values.
    """

    goaltending_data = {}

    for team, multiplier in weekly_multipliers.items():
        games = games_this_week.get(team, 0)

        # Estimate wins based on multiplier (inverse relation)
        projected_wins = games / multiplier
        projected_ot_losses = games * avg_ot_loss_freq  # You can adjust or use team-specific data
        projected_shutouts = games * avg_shutout_freq   # You can adjust or use team-specific data

        total_points = (projected_wins * win_points) + (projected_ot_losses * ot_loss_points) + (projected_shutouts * shutout_bonus)
        
        goaltending_data[team] = (total_points,games)

    return goaltending_data

# Using the function
goaltending_data = estimate_team_goaltending_points(weekly_multipliers, games_this_week, goaltending_win, ot_so_loss, shutout)

# Convert to DataFrame
df = pd.DataFrame(goaltending_data.values(), index=goaltending_data.keys(), columns=['proj_fantasy_pts', 'games_this_week'])

# Reset index to have the teams as a separate column
df = df.reset_index().rename(columns={"index": "Team"})

# Add the position
df["Position"] = "G" 
df['Injured'] = False
df['Player'] = df['Team']


goalie_df = df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

## Final DataFrame Ready for Optimization

In [20]:
# Add goalie df to player df
final_df = pd.concat([projections_df, goalie_df], ignore_index=True).round(2)

## Add player values
final_df = pd.merge(final_df, pv_df[["Team","Player","Position","pv"]], 
                     on=['Team',"Position",'Player'], 
                     how='left')

final_df.loc[final_df['Position'] =='G','pv'] = 0

final_df = final_df.sort_values(by=["pv","proj_fantasy_pts"], ascending=False)

## Optimize Lineup

In [21]:


## OPTIMIZATION RULES

"""
- 6 Forwards MUST be selected
- 4 Defense MUST be selected
- 2 team Goaltenders MUST be selected
- A team CANNOT go over the salary cap (MAX_COST)
- At most 4 players from a single team may be represented in the lineup
"""

MAX_COST = 63.00
MIN_COST = MAX_COST * 0.95
NUM_FORWARDS = 6
NUM_DEFENSEMEN = 4 
NUM_GOALIES = 2
MAX_PLAYERS_PER_TEAM = 4

final_df = final_df.dropna(subset=['proj_fantasy_pts', 'pv'])

def select_best_team(df,max_cost,min_cost,num_forwards, num_defensemen, num_goalies,max_players_per_team):
    # Create a linear programming problem
    prob = pulp.LpProblem("FantasyHockeyTeam", pulp.LpMaximize)

    # Create decision variables
    # Each player gets a binary decision variable
    player_vars = pulp.LpVariable.dicts("player", df.index, cat="Binary")

    # Objective: Maximize total fantasy points
    prob += pulp.lpSum(df['proj_fantasy_pts'][i] * player_vars[i] for i in df.index), "Total Fantasy Points"

    # Constraint: Stay within the salary cap
    prob += pulp.lpSum(df['pv'][i] * player_vars[i] for i in df.index) <= max_cost, "Total Salary Cost"
    prob += pulp.lpSum(df['pv'][i] * player_vars[i] for i in df.index) >= min_cost, "Minimum Salary Cost"
    
    # Positional constraints
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'F'].index) == num_forwards, "Number of Forwards"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'D'].index) == num_defensemen, "Number of Defensemen"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'G'].index) == num_goalies, "Number of Goalies"

    # Teams represented
    teams = df['Team'].unique()

    # Limit the number of players from each team directly
    for team in teams:
        team_player_indices = df[df['Team'] == team].index
        # Directly limit the number of players from each team
        prob += pulp.lpSum(player_vars[i] for i in team_player_indices) <= max_players_per_team, f"Max players from team {team}"

    
    # Solve the problem
    prob.solve(pulp.PULP_CBC_CMD())
#     prob.solve(pulp.GLPK_CMD())


    # Extract the selected players
    selected_players = [i for i in df.index if player_vars[i].varValue == 1]
    best_team = df.loc[selected_players]

    return best_team

# Assuming your dataframe is named final_df, and you have defined constants for max_cost, num_forwards, etc.
best_team = select_best_team(final_df, MAX_COST,MIN_COST,NUM_FORWARDS, NUM_DEFENSEMEN, NUM_GOALIES,MAX_PLAYERS_PER_TEAM)


# Calculate total projected points and total cost for the chosen players from best_team
total_projected_points = sum(best_team.proj_fantasy_pts)
total_cost = sum(best_team.pv)

# Displaying the results

# Create a dataframe with the values
cost = pd.DataFrame({'total_proj_pts': [total_projected_points], 'total_cost': [total_cost]})

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kellen/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/47a65144b3f145d69f4330e817bbc0f8-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/47a65144b3f145d69f4330e817bbc0f8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 42 COLUMNS
At line 4248 RHS
At line 4286 BOUNDS
At line 4905 ENDATA
Problem MODEL has 37 rows, 618 columns and 2408 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 59.2005 - 0.00 seconds
Cgl0004I processed model has 35 rows, 606 columns (606 integer (600 of which binary)) and 1785 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.0333061
Cbc0038I Solution found of -59.2
Cbc0038I Cleaned solution of -59.2
Cbc0038I Before mini branch and bound, 602 integer

# Lineup

In [22]:
print(cost)
best_team.sort_values(by='Position')

   total_proj_pts  total_cost
0            59.2      59.901


Team  Injured               Player Position  \
348     Utah Hockey Club    False    Mikhail Sergachev        D   
312         Dallas Stars    False       Miro Heiskanen        D   
193      Ottawa Senators    False        Thomas Chabot        D   
156    Vancouver Canucks    False         Quinn Hughes        D   
76          Dallas Stars    False      Jason Robertson        F   
8          Winnipeg Jets    False          Kyle Connor        F   
38         Winnipeg Jets    False       Nikolaj Ehlers        F   
2         Calgary Flames    False      Mikael Backlund        F   
45          Dallas Stars    False       Wyatt Johnston        F   
48          Dallas Stars    False      Logan Stankoven        F   
726        Winnipeg Jets    False        Winnipeg Jets        G   
743  Tampa Bay Lightning    False  Tampa Bay Lightning        G   

     games_this_week  proj_fantasy_pts     pv  
348                3              2.85  8.500  
312                3              3.48  8.450  
193                3              2.95  8.000  
156                3              3.43  7.850  
76                 3              4.40  7.750  
8                  4              5.36  7.143  
38                 4              4.31  6.000  
2                  3              4.51  4.500  
45                 3              4.80  0.894  
48                 3              4.68  0.814  
726                4              9.60  0.000  
743                3              8.83  0.000

## Captains

In [23]:
# Find the top players by position based on 'proj_fantasy_pts'
top_players = best_team.loc[best_team.groupby('Position')['proj_fantasy_pts'].idxmax()]

top_players

Team  Injured          Player Position  games_this_week  \
312   Dallas Stars    False  Miro Heiskanen        D                3   
8    Winnipeg Jets    False     Kyle Connor        F                4   
726  Winnipeg Jets    False   Winnipeg Jets        G                4   

     proj_fantasy_pts     pv  
312              3.48  8.450  
8                5.36  7.143  
726              9.60  0.000

In [24]:
data = [{'Team': key, 'Multiplier': value} for key, value in weekly_multipliers.items()]
multipliers = pd.DataFrame(data)
multipliers.sort_values(by="Multiplier").round(2)

Team  Multiplier
21    Tampa Bay Lightning        0.70
22        Edmonton Oilers        0.72
3        Florida Panthers        0.77
0         St. Louis Blues        0.79
11     Colorado Avalanche        0.79
1          Minnesota Wild        0.80
13    Washington Capitals        0.80
23         Calgary Flames        0.86
10    Pittsburgh Penguins        0.86
4           Winnipeg Jets        0.87
12    Carolina Hurricanes        0.89
28       New York Rangers        0.89
29      New Jersey Devils        0.90
27   Vegas Golden Knights        0.91
18      Detroit Red Wings        0.93
8      Chicago Blackhawks        0.93
2       Los Angeles Kings        0.94
5     Nashville Predators        0.95
25          Anaheim Ducks        0.95
30          Boston Bruins        0.96
31    Toronto Maple Leafs        0.96
19         Buffalo Sabres        0.98
16         Seattle Kraken        1.00
20     New York Islanders        1.00
17        Ottawa Senators        1.05
26    Philadelphia Flyers        1.05
15     Montreal Canadiens        1.09
24        San Jose Sharks        1.12
6       Vancouver Canucks        1.17
14  Columbus Blue Jackets        1.19
9        Utah Hockey Club        1.19
7            Dallas Stars        1.33

In [25]:
games_this_week

{'St. Louis Blues': 3,
 'Minnesota Wild': 3,
 'Los Angeles Kings': 3,
 'Florida Panthers': 3,
 'Winnipeg Jets': 4,
 'Nashville Predators': 3,
 'Vancouver Canucks': 3,
 'Dallas Stars': 3,
 'Chicago Blackhawks': 2,
 'Utah Hockey Club': 3,
 'Pittsburgh Penguins': 3,
 'Colorado Avalanche': 3,
 'Carolina Hurricanes': 3,
 'Washington Capitals': 3,
 'Columbus Blue Jackets': 2,
 'Montreal Canadiens': 3,
 'Seattle Kraken': 3,
 'Ottawa Senators': 3,
 'Detroit Red Wings': 3,
 'Buffalo Sabres': 3,
 'New York Islanders': 3,
 'Tampa Bay Lightning': 3,
 'Edmonton Oilers': 1,
 'Calgary Flames': 3,
 'San Jose Sharks': 3,
 'Anaheim Ducks': 3,
 'Philadelphia Flyers': 3,
 'Vegas Golden Knights': 3,
 'New York Rangers': 3,
 'New Jersey Devils': 3,
 'Boston Bruins': 3,
 'Toronto Maple Leafs': 2}

In [26]:
injuries_df

Player Position      Updated       Injury  \
0    Urho Vaakanainen        D  Fri, Nov 22   Upper Body   
1        Brock McGinn       LW  Fri, Nov 22   Lower Body   
2      Mason McTavish        C  Fri, Nov 22   Upper Body   
3        Robby Fabbri        C  Fri, Nov 15         Knee   
4          Cam Fowler        D  Fri, Nov 15   Upper Body   
..                ...      ...          ...          ...   
84      Alex Ovechkin       LW  Thu, Nov 21         Calf   
85       Sonny Milano       LW  Wed, Nov 20   Upper Body   
86         T.J. Oshie       RW  Tue, Sep 24         Back   
87  Nicklas Backstrom        C  Tue, Apr 30          Hip   
88      Logan Stanley        D  Mon, Nov 18  Undisclosed   

                                   Injury Status        Team  
0       Expected to be out until at least Nov 25     Anaheim  
1       Expected to be out until at least Nov 25     Anaheim  
2   IR. Expected to be out until at least Nov 25     Anaheim  
3   IR. Expected to be out until at least Dec 28     Anaheim  
4   IR. Expected to be out until at least Nov 29     Anaheim  
..                                           ...         ...  
84  IR. Expected to be out until at least Dec 16  Washington  
85   IR. Expected to be out until at least Dec 3  Washington  
86  IR. Expected to be out until at least Sep 15  Washington  
87   IR. Expected to be out until at least Jul 1  Washington  
88  IR. Expected to be out until at least Nov 27    Winnipeg  

[89 rows x 6 columns]

In [27]:
team_df = projections_df[projections_df["Team"] == 'Edmonton Oilers']
team_df

Team  Injured               Player Position  games_this_week  \
3    Edmonton Oilers    False       Connor McDavid        F                1   
49   Edmonton Oilers    False       Leon Draisaitl        F                1   
69   Edmonton Oilers     True     Viktor Arvidsson        F                1   
83   Edmonton Oilers     True           Zach Hyman        F                1   
107  Edmonton Oilers    False         Jeff Skinner        F                1   
127  Edmonton Oilers    False  Ryan Nugent-Hopkins        F                1   
150  Edmonton Oilers    False     Vasily Podkolzin        F                1   
168  Edmonton Oilers    False        Adam Henrique        F                1   
270  Edmonton Oilers    False      Kasperi Kapanen        F                1   
283  Edmonton Oilers    False        Evan Bouchard        D                1   
296  Edmonton Oilers    False       Mattias Ekholm        D                1   
299  Edmonton Oilers     True        Darnell Nurse        D                1   
321  Edmonton Oilers    False          Corey Perry        F                1   
328  Edmonton Oilers    False         Connor Brown        F                1   
431  Edmonton Oilers    False          Brett Kulak        D                1   
518  Edmonton Oilers    False          Ty Emberson        D                1   
525  Edmonton Oilers    False         Troy Stecher        D                1   
622  Edmonton Oilers    False      Mattias Janmark        F                1   
625  Edmonton Oilers     True       Travis Dermott        D                1   
637  Edmonton Oilers    False           Derek Ryan        F                1   
678  Edmonton Oilers    False         Joshua Brown        D                1   
694  Edmonton Oilers    False           Noah Philp        F                1   

     proj_fantasy_pts  
3            1.275216  
49           0.929395  
69           0.000000  
83           0.000000  
107          0.554755  
127          0.684438  
150          0.468300  
168          0.468300  
270          0.345821  
283          0.677233  
296          0.554755  
299          0.000000  
321          0.331412  
328          0.374640  
431          0.338617  
518          0.151297  
525          0.223343  
622          0.136888  
625          0.000000  
637          0.050432  
678               NaN  
694               NaN

In [28]:
goalie_df

Team  Injured                 Player Position  \
0         St. Louis Blues    False        St. Louis Blues        G   
1          Minnesota Wild    False         Minnesota Wild        G   
2       Los Angeles Kings    False      Los Angeles Kings        G   
3        Florida Panthers    False       Florida Panthers        G   
4           Winnipeg Jets    False          Winnipeg Jets        G   
5     Nashville Predators    False    Nashville Predators        G   
6       Vancouver Canucks    False      Vancouver Canucks        G   
7            Dallas Stars    False           Dallas Stars        G   
8      Chicago Blackhawks    False     Chicago Blackhawks        G   
9        Utah Hockey Club    False       Utah Hockey Club        G   
10    Pittsburgh Penguins    False    Pittsburgh Penguins        G   
11     Colorado Avalanche    False     Colorado Avalanche        G   
12    Carolina Hurricanes    False    Carolina Hurricanes        G   
13    Washington Capitals    False    Washington Capitals        G   
14  Columbus Blue Jackets    False  Columbus Blue Jackets        G   
15     Montreal Canadiens    False     Montreal Canadiens        G   
16         Seattle Kraken    False         Seattle Kraken        G   
17        Ottawa Senators    False        Ottawa Senators        G   
18      Detroit Red Wings    False      Detroit Red Wings        G   
19         Buffalo Sabres    False         Buffalo Sabres        G   
20     New York Islanders    False     New York Islanders        G   
21    Tampa Bay Lightning    False    Tampa Bay Lightning        G   
22        Edmonton Oilers    False        Edmonton Oilers        G   
23         Calgary Flames    False         Calgary Flames        G   
24        San Jose Sharks    False        San Jose Sharks        G   
25          Anaheim Ducks    False          Anaheim Ducks        G   
26    Philadelphia Flyers    False    Philadelphia Flyers        G   
27   Vegas Golden Knights    False   Vegas Golden Knights        G   
28       New York Rangers    False       New York Rangers        G   
29      New Jersey Devils    False      New Jersey Devils        G   
30          Boston Bruins    False          Boston Bruins        G   
31    Toronto Maple Leafs    False    Toronto Maple Leafs        G   

    games_this_week  proj_fantasy_pts  
0                 3          7.932000  
1                 3          7.829061  
2                 3          6.652026  
3                 3          8.100000  
4                 4          9.600000  
5                 3          6.624862  
6                 3          5.448000  
7                 3          4.800000  
8                 2          4.499373  
9                 3          5.352000  
10                3          7.296000  
11                3          7.858763  
12                3          7.059370  
13                3          7.778993  
14                2          3.568000  
15                3          5.821251  
16                3          6.300000  
17                3          6.027003  
18                3          6.777273  
19                3          6.451812  
20                3          6.300000  
21                3          8.832000  
22                1          2.876000  
23                3          7.316886  
24                3          5.664000  
25                3          6.600000  
26                3          6.007317  
27                3          6.900000  
28                3          7.024118  
29                3          6.931924  
30                3          6.581853  
31                2          4.369776